In [ ]:
import numpy as np
from algorithms import steepest_descent, conjugate_gradient, secant, Finite_Difference, armijo
from cost_functions import V_a, gradV_a, V_b, gradV_b
from numpy.linalg import norm, eig
from functools import partial


In [16]:
def augmented_lagrangian(x0, 
                         cost_function, 
                         equality_constraints = None, 
                         inequality_constraints = None,
                         threshold = 1e-8, 
                         sigma_max = 1e8, 
                         track_history = False, 
                         log = False):

    def phi(x):
        cost = cost_function(x)
        lambda_eq = lambd[:num_ec , :]
        lambda_ineq = lambd[num_ec:num_c , :]
        sigma_eq = sigma[:num_ec , :]
        sigma_ineq = sigma[num_ec:num_c , :]
    
        for constraint in equality_constraints:
            ec = constraint(x)
            cost -= lambda_eq * ec + 0.5 * sigma_eq * ec **2
        for i,constraint in enumerate(inequality_constraints):
            ic = constraint(x)
            if ic <= sigma_ineq[i] / sigma_ineq[i]: 
                t = -lambda_ineq[i] * ic + 0.5 * sigma_ineq[i] * ic**2 
            else:
                t = -0.5 * lambda_ineq[i]**2 / sigma_ineq[i] 
            cost += t
        return(cost)

    x_history, V_history = [],[]
    num_ec = len(equality_constraints)
    num_ic = len(inequality_constraints)
    num_c = num_ec + num_ic

    lambd = np.zeros((num_c,1))
    sigma = np.ones((num_c,1))

    cost = 1e12 * sigma
    x = x0
    minimum = cost_function(x)
    x_history.append(x), V_history.append(minimum)
    j = 0
    while norm(cost) > threshold: 

        x,_ = steepest_descent(x,
                               phi,
                               None,
                               step_size = 'armijo',
                               threshold = threshold, 
                               max_iter = 1e3)
        previous_cost = cost
        inequality_cost = [cost(x) for cost in equality_constraints]
        equality_cost = [cost(x) for cost in inequality_constraints]
        cost = np.array(inequality_cost + equality_cost).reshape((-1,1))

        if any(sigma >= sigma_max):
            break
        if norm(cost, np.inf) > 0.25 * norm(previous_cost, np.inf):
            for i in range(num_c):
                if np.abs(cost[i]) > 0.25 * norm(previous_cost, np.inf):
                    sigma[i] *= 10
            continue
        lambd = lambd - (sigma * cost)
        minimum = cost_function(x)
        x_history.append(x), V_history.append(minimum)
        j += 1
        if log and j % 1e2 == 0: 
            print(f'x = {x}, V(x) = {minimum:.5f}')
    if track_history:
        return x, minimum, x_history, V_history
    else:
        return x, minimum
    

In [18]:



v = lambda x: -x[0] * x[1]
h1 = lambda x: -x[0] - x[1]**2 + 1
h2 = lambda x: x[0] + x[1]

cost_function = v
inequality_constraints = [h1, h2]
equality_constraints = []
gradient_function = None
x0 = np.zeros(2)

x, minimum = augmented_lagrangian(x0,
                    cost_function, 
                    equality_constraints, 
                    inequality_constraints,
                    log = True)
x, minimum 

KeyboardInterrupt: 